In [1]:
import pandas as pd
from dfply import *

from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

Facility_to_fips = pd.read_csv("./data/CMS_Facility_To_fips_crosswalk.csv", dtype=str )
Poverty = pd.read_csv("./data/PovertyEstimates.csv", dtype={'FIPS_code':str} )
Timely_Effective = pd.read_csv("./data/Timely_and_Effective_Care-Hospital.csv")

In [2]:
Poverty.head()

,FIPS_code,PCTPOVALL_2020
0,00000,11.9
1,01000,14.9
2,01001,11.2
3,01003,8.9
4,01005,25.5


In [3]:
Facility_to_fips.head()

,State,Facility ID,County Name,FIPS
0,AL,010001,HOUSTON,01069
1,AL,010005,MARSHALL,01095
2,AL,010006,LAUDERDALE,01077
3,AL,010007,COVINGTON,01039
4,AL,010008,CRENSHAW,01041


In [4]:
Timely_Effective.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Condition,Measure ID,Measure Name,Score,Sample,Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,Emergency Department,EDV,Emergency department volume,high,NaN,NaN,01/01/2019,12/31/2019
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,Preventive Care,IMM_3,Healthcare workers given influenza vaccination,99,4119,NaN,10/01/2020,03/31/2021
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,Emergency Department,OP_18b,Average (median) time patients spent in the em...,168,169,NaN,07/01/2020,12/31/2020
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,Emergency Department,OP_18c,Average (median) time patients spent in the em...,Not Available,Not Available,1,07/01/2020,12/31/2020
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,Heart Attack or Chest Pain,OP_2,Fibrinolytic Therapy Received Within 30 Minute...,Not Available,Not Available,5,07/01/2020,12/31/2020


In [5]:
Timely_Effective_2 = (Timely_Effective
>> select(X['Facility ID'], X.State, X['County Name'], X.Condition, X['Measure Name'], X.Score)
>> filter_by(X.Condition == "Emergency Department")
>> filter_by((X['Measure Name'] == ("Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better")) |
(X['Measure Name'] == "Left before being seen"))
>> select(X['Facility ID'], X.State, X['County Name'], X['Measure Name'], X.Score)
>> mutate(Score = pd.to_numeric(Timely_Effective['Score'], errors='coerce').fillna(0).astype(int))
>> mutate(Score = X.Score.astype('float'))
>> spread(X["Measure Name"], X.Score)
>> mutate(Avg_median_time_spent_in_emergency_department = X['Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better'])
>> drop(X['Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better'])
)
Timely_Effective_2

,Facility ID,State,County Name,Left before being seen,Avg_median_time_spent_in_emergency_department
0,010001,AL,Houston,3.0,168.0
1,010005,AL,Marshall,2.0,139.0
2,010006,AL,Lauderdale,1.0,182.0
3,010007,AL,Covington,2.0,119.0
4,010008,AL,Crenshaw,0.0,126.0
...,...,...,...,...,...
4713,670260,TX,Denton,0.0,184.0
4714,670266,TX,Harris,0.0,0.0
4715,670267,TX,Montgomery,0.0,0.0
4716,670280,TX,Harris,0.0,0.0
